In [1]:
import gmsh
import numpy as np
from importlib.machinery import SourceFileLoader

el metodo initialize 'abre' el programa.

In [2]:
gmsh.initialize()

'armamos' el modelo que queremos modificar:

In [3]:
modelname = 'chapa_agujero'

In [4]:
gmsh.model.add(modelname)

In [5]:
lc = 2
R = 1   # radio del agujero
L = 10

Definimos las coordenadas de los puntos

In [6]:
P  = [
     gmsh.model.geo.addPoint(0, 0, 0, lc/10),
     gmsh.model.geo.addPoint(R, 0, 0, lc/3),
     gmsh.model.geo.addPoint(L, 0, 0, lc) ,
     gmsh.model.geo.addPoint(L, L/2, 0, lc) ,
     gmsh.model.geo.addPoint(0, L/2, 0, lc/2) ,
     gmsh.model.geo.addPoint(0, R, 0, lc/3)
]

Luego definimos las lineas

In [7]:
L = [ 
 gmsh.model.geo.addLine(P[1],P[2]),
 gmsh.model.geo.addLine(P[2],P[3]),
 gmsh.model.geo.addLine(P[3],P[4]),
 gmsh.model.geo.addLine(P[4],P[5]),
 gmsh.model.geo.addCircleArc(P[5],P[0],P[1])
]

Las curvas nos van a servir para definir los bordes de nuestro modelo.

In [8]:
C1 = gmsh.model.geo.addCurveLoop(L)

Y las superficies:

In [9]:
S1 = gmsh.model.geo.addPlaneSurface([C1])

le avisamos que ya estan todos los elementos geométricos:

In [10]:
gmsh.model.geo.synchronize()

In [11]:
EmpotradoX = gmsh.model.addPhysicalGroup(1, [L[3]])
gmsh.model.setPhysicalName(1,EmpotradoX,'EmpotradoX')

EmpotradoY = gmsh.model.addPhysicalGroup(1,[ L[0]])
gmsh.model.setPhysicalName(1,EmpotradoY,'Empotradoy')

Traccionado = gmsh.model.addPhysicalGroup(1, [L[1]])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado')

Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')

In [12]:
gmsh.model.mesh.generate(2)

# El objeto mesh

El mallado es un objeto del cual podemos recuperar la información necesaria para interactuar con nuestro motor de elementos finitos. Por ejemplo para la informacion de los nodos:

In [13]:
NodeInfo = gmsh.model.mesh.get_nodes_for_physical_group(2,Superficie)

NumeroNodos = NodeInfo[0].shape[0]

In [14]:
MN = NodeInfo[1].reshape(NumeroNodos , 3)

## Elementos

Podemos ser un poco más precabios y tomar solo los triángulos, que son los que nos interesan para la guía 

In [15]:
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)

la matriz de conectividad también hay que reformatear para tener lo que estamos acostumbrados.

In [16]:
MC = ELEMENTS.reshape([ETAGS.shape[0],3])

# Condiciones de contorno

Con las definiciones de los Physical Groups, podemos sacar los índices de los nodos que estan empotrados o traccionados. Necesitamos definir una 'entidad' para los elements Traccionados:

In [17]:
entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])

Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)

Con los elementos líneas traccionadas podemos calcular las longitudes y distribuir la fuerza externa. 

In [18]:
Longitudes = np.abs( MN[Ltraccionada[:,0]-1,1] - MN[Ltraccionada[:,1]-1,1] )

Ahora puedo calcular las fuerzas:

In [19]:
Fuerzas = np.zeros((2*NumeroNodos,1))
espesor = 1
tension = 1000 #Pa

for l, linea in enumerate(Ltraccionada):
    Flocal = np.array([[1],[1]])*tension*espesor*Longitudes[l]/2
    n1 = linea[0]
    n2 = linea[1]
    #print(Flocal)
    Fuerzas[ np.array([2*(n1-1), 2*(n2-1)], dtype=int)] += Flocal
Desplazamientos = np.zeros((2*NumeroNodos,1))

Exy = np.hstack( (Fuerzas.reshape(NumeroNodos, 2) , np.zeros((NumeroNodos, 1))) )

In [20]:
fext = gmsh.view.add('FuerzasExternas')
Fext = gmsh.view.addModelData(fext,0,modelname,'NodeData',NodeInfo[0],Exy)

# Empotrados

Para definir los nodos empotrados necesito solamente los índices de los nodos en dicho physical group

In [21]:
NodosEmpotradosX = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpotradoX)
NodosEmpotradosY = gmsh.model.mesh.get_nodes_for_physical_group(1,EmpotradoY)

Con eso ya puedo calcular los valores de los índices de los nodos que resultarán empotrados.

In [22]:
s = []
for n, nodo in enumerate(NodosEmpotradosX[0]):
    s.append(
        2*(nodo-1)
    )
for n, nodo in enumerate(NodosEmpotradosY[0]):
    s.append(
        2*(nodo-1)+1
    )

s = np.array(s).astype(int).ravel()
r = np.linspace(0, 2*(NumeroNodos-1)+1, 2*NumeroNodos).astype(int)
r = np.delete( r, s )

# Nos divertimos un rato

In [23]:
#import matplotlib.pyplot as plt
#from matplotlib import quiver
#
#plt.style.use('default')
#plt.rc('figure',figsize=(15,10))
#
##plt.plot(MN[:,0],MN[:,1],'ok')
#
##plt.plot(MNE[:,0], MNE[:,1], label='Empotrados', lw = 5)
##plt.plot(MNT[:,0], MNT[:,1], label='Traccionados', lw = 5)
#fig = plt.figure()
#ax = fig.add_axes([0.1, 0.2, 0.5, 0.6])
#ax.triplot(MN[:,0],MN[:,1],MC-1, )
##ax.plot(MN[:,0],MN[:,1],'o')
#ax.quiver( MN[:,0], MN[:,1], Exy[:,0], Exy[:,1], linewidth=5, units='width', scale=10000)
##ax.set_xlim(-1, L+(Fx/500).max())

# Agregar Resulatados

In [24]:
# import mefmods as mef
mef = SourceFileLoader('mef','mefmods.py').load_module()

In [25]:
ETYPES = 2*np.ones(len(MC)).astype(int)

nu = 0.3

E = 210E9

MP = np.hstack(
        (
            np.ones((len(MC), 1)),
            np.ones((len(MC), 1))*nu,
            np.ones((len(MC), 1))*E
            )
        )

In [26]:
K = mef.ensamble(MC-1, MN, MP, 2, ETYPES, 'Chapa-Asimetrica-2021')

In [27]:
s.sort()

In [31]:
#Desplazamientos[r] = 
Desplazamientos, FResultado = mef.resolvermef(r,s,K,Desplazamientos[s], Fuerzas[r], 'Chapa-Asimetrica-2021')

In [32]:
# import mefmods as mef
mef = SourceFileLoader('mef','mefmods.py').load_module()
sigma = mef.getstress2(MC, ETYPES, MP, MN, Desplazamientos)

In [33]:
SA = 0.5*(sigma[:,0]+sigma[:,1])
SB = np.sqrt(SA**2+ sigma[:,2]**2)
SIGMA_MAX = SA+SB
SIGMA_MIN = SA-SB

In [34]:
Rxy = np.hstack( (FResultado.reshape(NumeroNodos, 2) , np.zeros((NumeroNodos, 1))) )

In [35]:
Dxy = np.hstack( (Desplazamientos.reshape(NumeroNodos, 2) , np.zeros((NumeroNodos, 1))) )

In [43]:
SIGMA_AVE_MAX = np.zeros([NumeroNodos,1])

In [44]:
SIGMA_AVE_MIN = np.zeros([NumeroNodos,1])

In [45]:
for node in range(NumeroNodos):
    SIGMA_AVE_MAX[node] = SIGMA_MAX[ ETAGS[ (MC == node+1).any(axis=1) ] - ETAGS.min() ].mean()

In [46]:
for node in range(NumeroNodos):
    SIGMA_AVE_MIN[node] = SIGMA_MIN[ ETAGS[ (MC == node+1).any(axis=1) ] - ETAGS.min() ].mean()

# Guardo los resultados 

In [36]:
despview = gmsh.view.add('Desplazamientos')
Desp = gmsh.view.addModelData(despview,0,modelname,'NodeData',NodeInfo[0],Dxy)

In [37]:
forview = gmsh.view.add('Fuerzas')
Fza = gmsh.view.addModelData(forview,0,modelname,'NodeData',NodeInfo[0], Rxy)

In [38]:
tenmaxview = gmsh.view.add('Tension Maxima')
tenmax = gmsh.view.addModelData(tenmaxview,0,modelname,'ElementData',ETAGS, SIGMA_MAX.reshape(-1,1))

In [39]:
tenminview = gmsh.view.add('Tension Mínima')
tenmax = gmsh.view.addModelData(tenminview,0,modelname,'ElementData',ETAGS, SIGMA_MIN.reshape(-1,1))

In [40]:
tenxview = gmsh.view.add('Sigma_x')
tenx = gmsh.view.addModelData(tenxview,0,modelname,'ElementData',ETAGS, sigma[:,0].reshape(-1,1))

In [41]:
tenyview = gmsh.view.add('Sigma_y')
teny = gmsh.view.addModelData(tenyview,0,modelname,'ElementData',ETAGS, sigma[:,1].reshape(-1,1))

In [42]:
tencorte = gmsh.view.add('Tau_xy')
tenc = gmsh.view.addModelData(tencorte, 0,modelname,'ElementData',ETAGS, sigma[:,2].reshape(-1,1))

In [47]:
tenmaxave = gmsh.view.add('Sigma_max_ave')
tenmave = gmsh.view.addModelData(tenmaxave, 0,modelname,'NodeData',NodeInfo[0], SIGMA_AVE_MAX.reshape(-1,1))

In [48]:
tenminave = gmsh.view.add('Sigma_min_ave')
tenmiave = gmsh.view.addModelData(tenminave, 0,modelname,'NodeData',NodeInfo[0], SIGMA_AVE_MIN.reshape(-1,1))

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()